In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
rng = np.random.default_rng(42)

In [2]:
hpi_df = pd.read_csv("hpi.csv")
base_df = pd.read_csv("base.csv")

In [3]:
# Year FE
year_list = sorted(list(hpi_df['YEAR'].unique()))
year_fe = {}
for idx, row in hpi_df.iterrows():
    year_fe[row['YEAR']] = np.log(row['HPI']/100)

In [4]:
# Zip FE
zip_list = range(8001, 8022)
zip_fe = {}
zip_rural = {}
for z in zip_list:
    zip_rural[z] = rng.integers(0,2)
    if zip_rural[z]==0:
        zip_fe[z] = rng.uniform(low=0,high=1.5)
    else:
        zip_fe[z] = rng.uniform(low=-0.5,high=0)

In [5]:
# Create fake years and zipcodes
df = base_df.copy()
df['sale_year'] = rng.choice(year_list, size=len(df))
df['zipcode'] = rng.choice(zip_list, size=len(df))

In [6]:
# Modify year built so that year built can't be larger than sale_year
df['age_at_sale'] = 2022 - df['year_built']
df['year_built'] = df['sale_year'] - df['age_at_sale']
df['built_before_1977'] = (df['year_built']<1977)*1

In [7]:
# Attach FEs to the dataframe
df['year_fe'] = 0.0
df['zip_fe'] = 0.0
df['zip_rural'] = 0
for idx, row in df.iterrows():
    df.loc[idx,'year_fe'] = year_fe[row['sale_year']]
    df.loc[idx,'zip_fe'] = zip_fe[row['zipcode']]
    df.loc[idx,'zip_rural'] = zip_rural[row['zipcode']]

In [8]:
# Simulate prices
df['log_price'] = np.log(500) + \
    0.67*np.log(df['sq_ft']) + \
    0.23*np.log(df['lot_sq_ft']) + \
    0.09*(df['num_bedrooms']) + \
    0.15*(df['num_bathrooms']) + \
    -0.08*(df['built_before_1977']) + \
    0.11*df['has_garage'] + \
    0.17*df['has_view'] + \
    df['year_fe'] + \
    df['zip_fe'] + \
    0.23*df['zip_rural']*(df['sale_year']>=2020) + \
    rng.normal(loc=0, scale=0.1, size=len(df))

df['sale_price'] = np.round(np.exp(df['log_price'])/1000)*1000

In [9]:
# Output dataframes
df = df.sort_values(by='sale_year', ascending=True).reset_index(drop=True)
df['sale_id'] = df.index

sale_df = df[['sale_id', 'sale_price', 'sale_year', 'zipcode', 'sq_ft', 'lot_sq_ft', 'num_bedrooms', 'num_bathrooms', 'built_before_1977', 'has_garage', 'has_view']]
sale_df.to_csv("sales.csv", header=True, index=False)

zip_df = df[['zipcode', 'zip_rural']].drop_duplicates().rename(columns={'zip_rural':'is_rural'}).sort_values(by='zipcode',ascending=True).reset_index(drop=True)
zip_df.to_csv("zips.csv", header=True, index=False)
